# sklearn/문장 특징 추출과 유사도 측정

https://blog.breezymind.com/2018/03/02/sklearn-feature_extraction-text-2/

## : Scikit-Learn의 text 특징추출과 벡터화를 통한 유사도 측정(CountVecotrizer, TfidVectorizer, HashingVectorizer)

### 문장 특징추출 준비

In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
np.random.seed(0) # 난수 발생 시드 설정 --> 무엇을 의미?
from konlpy.tag import *
okt = Okt()
han = Hannanum()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [17]:
#tokenizer : 문장에서 색인어 추출을 위해 명사, 동사, 알파벳, 숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun", "Alpha", "Verb", "Number"], storpword=[]):
    return [
        word for word, tag in han.pos(
        raw
        #normalize=True,
        #stemming=True
        )
        if len(word) > 1 and tag in pos and word not in stopword
    ]


#테스트 문장
rawdata = [
    '아시아의 고산지대인 파미르, 텐샨,히말라야 의 자연과 사람들의 삶을 알아본다.',
    '중앙아시아의 거대한 호수 ‘아랄해’, 지금은 거의 말라버려 20세기 최악의 환경재앙이라 불리는 곳입니다.',
    '2부는 이 아랄해로 들어오는 두 물줄기 ‘아무다리아강’과 ‘시르다리아강’을 따라 그 발원지를 찾아가는 여정입니다.',
    '<주요 에피소드> 아무다리아강의 발원지, 파미르고원 대륙빙하 중 최대 규모인 파미르의 페드첸코 빙하에서부터 녹아 흐르는 물은 판즈강을 거쳐 아무다리아 강이 된다.',
    '타지키스탄과 아프가니스탄의 국경을 이루는 강인 판즈강을 따라 가면 쉽게 볼 수 없는 아프가니스탄 사람들의 모습을 볼 수 있다.',
    ' 또 빙하부터 습지대까지 식생의 변화와, 환경의 변화를 살펴보고 강에 기대어 살아가는 사람들의 삶을 소개한다.   시르다리아강의 발원지는 텐샨산맥 파미르 고원에 비해 고도가 낮은 텐샨산맥은 아름다운 풍경을 자랑한다.',
    ' 초록으로 뒤덮인 산과 색색의 꽃들은 화려한 텐샨을 만든다. 시르다리야강의 발원지인 텐샨의 빙하는 어떤 모습인지 소개한다. 자일로(여름 초목지)에 유르트를 짓고 양을 치는 키르기 유목민, 텐샨산맥의 야생동물을 만난다. '
]

### CountVectorizer

In [18]:
vectorize = CountVectorizer(
    tokenizer=tokenizer, 
    min_df=2    # 예제로 보기 좋게 1번 정도만 노출되는 단어들은 무시하기로 했다
                # min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 무시
                # min_df = 10 : 문서에 10개 미만으로 나타나는 단어 무시
                # max_df = 0.80 : 문서의 80% 이상에 나타나는 단어 무시
                # max_df = 10 : 10개 이상의 문서에 나타나는 단어 무시
)
 
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 Document Term Matrix(이하 DTM) 을 리턴한다
X = vectorize.fit_transform(rawdata)
 
print(
    'fit_transform, (sentence {}, feature {})'.format(X.shape[0], X.shape[1])
)
# fit_transform, (sentence 5, feature 7)
 
print(type(X))
# <class 'scipy.sparse.csr.csr_matrix'>
 
print(X.toarray())
 
# [[0, 1, 2, 0, 0, 0, 1],
# [0, 1, 1, 0, 0, 0, 2],
# [1, 0, 0, 2, 1, 1, 0],
# [1, 0, 0, 1, 0, 0, 0],
# [0, 0, 0, 3, 1, 1, 0]]
 
# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()
 
 

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [19]:
import pandas as pd
pd.options.mode.chained_assignment = None
 
import numpy as np
np.random.seed(0)
 
from konlpy.tag import Twitter
twitter = Twitter()
 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
 
# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

# 테스트 문장
rawdata = [
    '남북 고위급회담 대표단 확정..남북 해빙모드 급물살',
    '[남북 고위급 회담]장차관만 6명..판 커지는 올림픽 회담',
    
    '문재인 대통령과 대통령의 영부인 김정숙여사 내외의 동반 1987 관람 후 인터뷰',
    '1987 본 문 대통령.."그런다고 바뀌나? 함께 하면 바뀐다"',
    
    '이명박 전 대통령과 전 대통령의 부인 김윤옥 여사, 그리고 전 대통령의 아들 이시형씨의 동반 검찰출석이 기대됨'
]

C:\Users\dayoi\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [20]:
vectorize = CountVectorizer(
tokenizer=tokenizer, 
    min_df=2    # 예제로 보기 좋게 1번 정도만 노출되는 단어들은 무시하기로 했다
                # min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 무시
                # min_df = 10 : 문서에 10개 미만으로 나타나는 단어 무시
                # max_df = 0.80 : 문서의 80% 이상에 나타나는 단어 무시
                # max_df = 10 : 10개 이상의 문서에 나타나는 단어 무시
)
 
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 Document Term Matrix(이하 DTM) 을 리턴한다
X = vectorize.fit_transform(rawdata)
 
print(
    'fit_transform, (sentence {}, feature {})'.format(X.shape[0], X.shape[1])
)
# fit_transform, (sentence 5, feature 7)
 
print(type(X))
# <class 'scipy.sparse.csr.csr_matrix'>
 
print(X.toarray())
 
# [[0, 1, 2, 0, 0, 0, 1],
# [0, 1, 1, 0, 0, 0, 2],
# [1, 0, 0, 2, 1, 1, 0],
# [1, 0, 0, 1, 0, 0, 0],
# [0, 0, 0, 3, 1, 1, 0]]
 
# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()

fit_transform, (sentence 5, feature 7)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 1 2 0 0 0 1]
 [0 1 1 0 0 0 2]
 [1 0 0 2 1 1 0]
 [1 0 0 1 0 0 0]
 [0 0 0 3 1 1 0]]
